In [1]:
#pip install "numpy<2.0" 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm

import torch
from torch import nn

import sys
import os

# Determine the path to the parent directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_dir)

import ipynb.fs
from ipynb.fs.full.model import VariationalAutoEncoder

from torchvision import transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, RandomSampler
from torchvision.utils import save_image 

In [2]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
INPUT_DIM = 256
INIT_DIM = 8
LATENT_DIM = 3
BATCH_SIZE = 1
LR_RATE = 3e-4
KERNEL_SIZE = 4

In [3]:
# Dataset Loading
data_path = '../dataset' # setting path
transform = transforms.Compose([transforms.Resize((INPUT_DIM, INPUT_DIM)),   # sequence of transformations to be done
                                transforms.Grayscale(num_output_channels=1), # on each image (resize, greyscale,
                                transforms.ToTensor()])                      # convert to tensor)

dataset = datasets.ImageFolder(root=data_path, transform=transform) # read data from folder

train_loader = DataLoader(dataset=dataset, batch_size=BATCH_SIZE, shuffle=True) # create dataloader object

model = VariationalAutoEncoder(init_dim=INIT_DIM, latent_dim=LATENT_DIM, kernel_size=KERNEL_SIZE).to(DEVICE) # initializing model object

optimizer = torch.optim.Adam(model.parameters(), lr=LR_RATE) # defining optimizer
loss_fn = nn.BCELoss(reduction='sum') # define loss function

In [4]:
model = VariationalAutoEncoder(init_dim=INIT_DIM, latent_dim=LATENT_DIM, kernel_size=KERNEL_SIZE)
model.load_state_dict(torch.load('../models/model_256x'))
model.eval()

VariationalAutoEncoder(
  (enc1): Conv2d(1, 8, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (enc2): Conv2d(8, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (enc3): Conv2d(16, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (enc4): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
  (fc1): Linear(in_features=14400, out_features=128, bias=True)
  (fc_mu): Linear(in_features=128, out_features=3, bias=True)
  (fc_log_var): Linear(in_features=128, out_features=3, bias=True)
  (fc2): Linear(in_features=3, out_features=256, bias=True)
  (dec1): ConvTranspose2d(256, 256, kernel_size=(4, 4), stride=(1, 1))
  (dec2): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (dec3): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (dec4): ConvTranspose2d(64, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (dec5): ConvTranspose2d(32, 8, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (dec6): ConvTranspose2d

In [5]:
def decode(x, y, z):
    out = model.decode(torch.tensor([x, y, z]).float())
    out = out.view(-1, 1, 256, 256).detach().numpy()
    img = out[0][0]
    img = (img*255).astype(np.uint8)
    return img

In [13]:
import tkinter as tk
from PIL import Image, ImageTk
from tkinter import messagebox, simpledialog
import os
import cv2

# Função para criar um quadrado com bordas
def create_square(canvas, x, y, size, border_size, border_color, fill_color, tag):
    canvas.create_rectangle(
        x, y, x + size, y + size,
        outline=border_color, width=border_size,
        fill=fill_color, tags=tag
    )
    
# Função para verificar se as coordenadas estão dentro do quadrado esquerdo
def is_within_left_square(x, y):
    left_square_x1 = padding
    left_square_y1 = (window_height - 256) // 2
    left_square_x2 = left_square_x1 + 256
    left_square_y2 = left_square_y1 + 256
    return left_square_x1 <= x <= left_square_x2 and left_square_y1 <= y <= left_square_y2

# Função para normalizar as coordenadas
def normalize_coordinates(x, y):
    # Intervalo original
    min_original = padding
    max_original = 256 + padding
    
    # Intervalo novo
    min_novo = -3
    max_novo = 3
    
    # Normalizar
    norm_x = ((x - min_original) * (max_novo - min_novo)) / (max_original - min_original) + min_novo
    norm_y = ((y - min_original) * (max_novo - min_novo)) / (max_original - min_original) + min_novo
    
    return norm_x, norm_y

# Função para desenhar um sinal de "+" na última coordenada escolhida
def draw_plus_sign(x, y):
    canvas.delete("plus_sign")  # Limpar o sinal anterior, se houver
    size = 10  # Tamanho do sinal de "+"
    canvas.create_line(x - size, y, x + size, y, fill="#2BD9C2", width=2, tags="plus_sign")
    canvas.create_line(x, y - size, x, y + size, fill="#2BD9C2", width=2, tags="plus_sign")
    
# Função para exibir as coordenadas x e y no canto da janela
def display_coordinates():
    global current_x, current_y, current_z
    text = f"x: {current_x:.2f}, y: {current_y:.2f}, z: {current_z:.2f}"
    canvas.delete("coordinates_text")  # Limpar o texto anterior, se houver
    canvas.create_text(padding, window_height - 25, anchor=tk.NW, text=text, fill="white", tags="coordinates_text", font=("Roboto", 12))

def set_plane(plane):
    global current_plane
    current_plane = plane
    display_coordinates()  # Atualizar a imagem com o plano selecionado

def create_plane_buttons():
#     # Cria um frame para os botões
#     btn_frame = tk.Frame(root, bg="black")
    
#     # Define a localização do frame na janela principal
#     btn_frame.place(x=window_width - 60, y=window_height - 25)
    
#     # Cria os botões e os adiciona ao frame
#     btn_xy = tk.Button(btn_frame, text="XY", command=lambda: set_plane("XY"), width = 1, height = 1)
#     btn_xy.pack(side=tk.LEFT)

#     btn_yz = tk.Button(btn_frame, text="YZ", command=lambda: set_plane("YZ"), width = 1, height = 1)
#     btn_yz.pack(side=tk.LEFT)

#     btn_xz = tk.Button(btn_frame, text="XZ", command=lambda: set_plane("XZ"), width = 1, height = 1)
#     btn_xz.pack(side=tk.LEFT)
    
    button_width = 2
    button_height = 1
    button_padding = 22  # Espaço entre os botões
    font_size = 6

    button_xy = tk.Button(root, text="XY", command=lambda: set_plane("XY"), width=button_width, height=button_height, font=("Arial", font_size))
    button_xy.place(x=window_width - 25 - 2*(button_width + button_padding), y=window_height - 22)

    button_yz = tk.Button(root, text="YZ", command=lambda: set_plane("YZ"), width=button_width, height=button_height, font=("Arial", font_size))
    button_yz.place(x=window_width - 25 - button_width - button_padding, y=window_height - 22)

    button_xz = tk.Button(root, text="XZ", command=lambda: set_plane("XZ"), width=button_width, height=button_height, font=("Arial", font_size))
    button_xz.place(x=window_width - 25, y=window_height - 22)
    
def process_coordinates(x, y):
    global current_plane
    global current_x, current_y, current_z
    
    # Desenhar um sinal de "+" na última coordenada escolhida no quadrado esquerdo
    draw_plus_sign(x, y)
    
    # Normalizar coordenadas para um range [-3, 3]
    norm_x, norm_y = normalize_coordinates(x, y)
    
    # Inicializar norm_z como 0
    norm_z = 0
    
    # Atualizar as coordenadas armazenadas com base no plano atual
    if current_plane == "XY":
        current_x, current_y = norm_x, norm_y
    elif current_plane == "YZ":
        current_y, current_z = norm_y, norm_x
    elif current_plane == "XZ":
        current_x, current_z = norm_x, norm_y
    
    # Adicionar as coordenadas à lista de pontos
    points.append((current_x, current_y, current_z))
    
    # Escrever coordenadas no app
    display_coordinates()
    
    # Gerar a imagem com base nas coordenadas normalizadas
    img = decode(norm_x, norm_y, norm_z)
    
    # Converter o array NumPy para uma imagem PIL
    img = Image.fromarray(img)
    
    # Desenhar a imagem no quadrado da direita
    draw_image_in_right_square(img)

    
# Função de callback para o evento de clique
def on_square_click(event):
    canvas = event.widget
    x = canvas.canvasx(event.x)
    y = canvas.canvasy(event.y)
    if is_within_left_square(x, y):
        process_coordinates(x, y)

# Função de callback para o evento de arrastar o mouse
def on_square_drag(event):
    canvas = event.widget
    x = canvas.canvasx(event.x)
    y = canvas.canvasy(event.y)
    if is_within_left_square(x, y):
        process_coordinates(x, y)
        
def on_square_release(event):
    global points
    point_count = len(points)
    response = messagebox.askyesno("Salvar pontos", f"Você arrastou sobre {point_count} pontos. Deseja salvar esses pontos?")
    if response:
        folder_name = simpledialog.askstring("Nome da Pasta", "Digite o nome da pasta onde deseja salvar os pontos:")
        if folder_name:
            os.makedirs(folder_name)
            try:
                for i,p in enumerate(points):
                    img = decode(p[0], p[1], p[2])
                    _, img = cv2.threshold(img, 150, 255, cv2.THRESH_BINARY)
                    plt.imsave(folder_name + f'/{i}.png', img, cmap='gray')
                messagebox.showinfo("Pontos salvos", "Os pontos foram salvos com sucesso.")
            except Exception as e:
                print(e)
                print('Ocorreu um erro ao salvar as imagens.')
        else:
            messagebox.showwarning("Nome inválido", "Nome da pasta não pode ser vazio.")
    else:
        messagebox.showinfo("Pontos não salvos", "Os pontos não foram salvos.")
    points = []  # Resetar a lista de pontos após o popup
    
# Função para desenhar uma imagem no quadrado da direita
def draw_image_in_right_square(image):
    # Converter a imagem para o formato que o Tkinter pode exibir
    tk_image = ImageTk.PhotoImage(image)
    
    # Limpar o canvas da direita antes de desenhar a nova imagem
    canvas.delete("right_image")
    
    # Desenhar a imagem no quadrado da direita
    canvas.create_image(x2, y1, anchor=tk.NW, image=tk_image, tags="right_image")
    
    # Manter uma referência à imagem para evitar que ela seja coletada pelo garbage collector
    canvas.image = tk_image
    
# Função para desenhar o grid no quadrado esquerdo
def draw_grid():
    # Intervalos para os eixos x e y
    interval = 256 // 6  # Dividir o quadrado de 256px em 6 intervalos
    
    # Desenhar as linhas verticais do grid
    for i in range(1, 6):
        if i == 3: continue
        x = padding + i * interval
        canvas.create_line(x, y1, x, y1 + 256, fill="#2BD9C2", dash=(2, 2), tags="grid")

    # Desenhar as linhas horizontais do grid
    for i in range(1, 6):
        if i == 3: continue
        y = y1 + i * interval
        canvas.create_line(padding, y, padding + 256, y, fill="#2BD9C2", dash=(2, 2), tags="grid")

    # Desenhar os eixos x e y
    canvas.create_line(padding + 128, y1, padding + 128, y1 + 256, fill="#2BD9C2", width=2, tags="grid")  # Eixo y
    canvas.create_line(padding, y1 + 128, padding + 256, y1 + 128, fill="#2BD9C2", width=2, tags="grid")  # Eixo x

# Configurações da janela
window_width = 542
window_height = 296
padding = 10
border_size = 5
border_color = "#2BD9C2"
fill_color = "black"

# Variáveis globais para armazenar as coordenadas atuais e a lista de pontos
current_plane = "XY"
current_x = 0
current_y = 0
current_z = 0
points = []  # Lista para armazenar os pontos

# Criar a janela principal
root = tk.Tk()
root.title("VAE latent space viewer")
root.geometry(f"{window_width}x{window_height}")
root.configure(bg="black")  # Definir o fundo da janela principal como preto

# Criar um canvas para desenhar os quadrados
canvas = tk.Canvas(root, width=window_width, height=window_height, bg="black")
canvas.pack()

# Coordenadas para o primeiro quadrado
x1 = padding
y1 = padding

# Coordenadas para o segundo quadrado
x2 = x1 + 256 + padding

# Criar os quadrados
create_square(canvas, x1, y1, 256, border_size, border_color, fill_color, "left_square")
create_square(canvas, x2, y1, 256, border_size, border_color, fill_color, "right_square")

# Desenhar o grid no quadrado esquerdo
draw_grid()

# Associar os eventos de clique e arrastar o mouse ao quadrado da esquerda
canvas.tag_bind("left_square", "<Button-1>", on_square_click)
canvas.tag_bind("left_square", "<B1-Motion>", on_square_drag)
canvas.tag_bind("left_square", "<ButtonRelease-1>", on_square_release)

# Inicializar a imagem no quadrado da direita com as coordenadas (0, 0)
process_coordinates(padding + 128, padding + 128)

# Criar os botões de seleção do plano
create_plane_buttons()

# Iniciar o loop principal da GUI
root.mainloop()